# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
#Load and Read CSV
vacation_path = "clean_city_data.csv"
vacation_data = pd.read_csv(vacation_path)

#Had to remove extra column because previous index was added as a column
vacation_data = vacation_data[['City', 'Lat', 'Lng', 'Max Temp', 'Humidity',
                                  'Cloudiness', 'Wind Speed', 'Country', 'Date']]
vacation_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Honningsvåg,70.98,25.97,50.00,100,90,2.24,NO,1595538826
1,Havøysund,71.00,24.66,50.00,100,90,2.24,NO,1595538818
2,Prince Rupert,54.32,-130.32,53.60,100,90,6.93,CA,1595538873
3,Penzance,50.12,-5.54,64.00,100,72,12.75,GB,1595538873
4,Tornio,65.85,24.15,55.40,100,90,6.93,FI,1595538832
...,...,...,...,...,...,...,...,...,...
542,Bam,29.11,58.36,86.14,13,0,7.34,IR,1595538859
543,Sakakah,29.97,40.21,96.80,12,44,9.17,SA,1595538544
544,Taoudenni,22.68,-3.98,103.51,11,0,4.16,ML,1595538852
545,Calama,-22.47,-68.93,60.80,10,0,18.34,CL,1595538860


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
# Configure gmaps
gmaps.configure(api_key= g_key)

# Store latitude and longitude in locations
locations = vacation_data[["Lat", "Lng"]].astype(float)

# Fill NaN values and convert to float
humidity = vacation_data['Humidity'].astype(float)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
#Max temperature lower than 80 degrees but higher than 70
vacation_data = vacation_data.loc[(vacation_data['Max Temp'] <80) & (vacation_data['Max Temp'] >70)]

#Zero cloudiness
vacation_data = vacation_data.loc[(vacation_data['Cloudiness'] == 0)]

#Wind speed less than 10 mph
vacation_data = vacation_data.loc[(vacation_data['Wind Speed'] < 10)]

vacation_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
181,Soyo,-6.13,12.37,71.73,82,0,8.12,AO,1595538843
216,Lata,40.16,-8.33,71.01,78,0,1.01,PT,1595538818
224,Marsá Maţrūḩ,31.35,27.25,77.00,78,0,7.47,EG,1595538865
311,Manokwari,-0.87,134.08,78.85,71,0,4.00,ID,1595538793
319,Lagoa,39.05,-27.98,70.07,70,0,4.25,PT,1595538855
399,Kiryat Gat,31.61,34.76,78.01,62,0,0.16,IL,1595538850
401,Netivot,31.42,34.59,78.01,62,0,0.83,IL,1595538859
423,Tongliao,43.61,122.27,73.74,58,0,4.32,CN,1595538738
439,Lingyuan,41.24,119.40,70.27,55,0,4.81,CN,1595538856
475,Naviraí,-23.07,-54.19,74.89,47,0,3.96,BR,1595538870


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
#set empty lists
hotel = []
hotel_lat = []
hotel_lng = []

#define search
target_search = 'hotels'

#define radius
radius = 5000

hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

for index, row in vacation_data.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params).json()
    
    try:
        hotel.append(response["results"][0]["name"])
        hotel_lat.append(response["results"][0]["geometry"]["location"]["lat"])
        hotel_lng.append(response["results"][0]["geometry"]["location"]["lng"])
    except (KeyError, IndexError):
        next

In [45]:
hotel_dict = {
    "Hotel Name": hotel,
    "Lat": hotel_lat,
    "Lng": hotel_lng
}

hotel_df = pd.DataFrame(hotel_dict)

hotel_df

,Hotel Name,Lat,Lng
0,Kinwica,-6.163769,12.351399
1,Magnolias Cottage,40.162512,-8.271962
2,Porto Matrouh Hotel,31.354344,27.237316
3,Swiss Belhotel Manokwari,-0.857170,134.067035
4,Graciosa Resort - Biosphere Island Hotel,39.085412,-27.997393
5,Desert Gat,31.603043,34.763726
6,Country Garden Phoenix Hotel Tongliao,43.619440,122.265737
7,汉庭酒店,41.243238,119.409787
8,Nav Park Hotel,-23.057488,-54.193132
9,Hotel King Saron Ξενοδοχείο,37.906377,23.011527


In [46]:
#Get city and country location of hotels
City = []
Country = []

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["address"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    response = requests.get(base_url, params).json()
    
    try:
        City.append(response["results"][0]["address_components"][2]["long_name"])
        Country.append(response["results"][0]["address_components"][3]["short_name"])
    except (KeyError, IndexError):
        next

In [54]:
#Turn City and Country to DataFrames
Country_df = pd.DataFrame(Country)
City_df = pd.DataFrame(City)

#Add City and Country Columns
hotel_df['City'] = City_df
hotel_df['Country'] = Country_df
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Kinwica,-6.163769,12.351399,Zaire,AO
1,Magnolias Cottage,40.162512,-8.271962,Coimbra,PT
2,Porto Matrouh Hotel,31.354344,27.237316,Matrouh Governorate,EG
3,Swiss Belhotel Manokwari,-0.857170,134.067035,Manokwari Barat,Manokwari Bar.
4,Graciosa Resort - Biosphere Island Hotel,39.085412,-27.997393,Açores,PT
5,Desert Gat,31.603043,34.763726,Kiryat Gat,Ashkelon
6,Country Garden Phoenix Hotel Tongliao,43.619440,122.265737,Tongliao Shi,Chaoyang Shi
7,汉庭酒店,41.243238,119.409787,Lingyuan Shi,Naviraí
8,Nav Park Hotel,-23.057488,-54.193132,Centro,GR
9,Hotel King Saron Ξενοδοχείο,37.906377,23.011527,Korinthia,Várzea da Palma


In [53]:
# base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
# response = requests.get(base_url, params).json()
# response

In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
locations

,Lat,Lng
0,-6.163769,12.351399
1,40.162512,-8.271962
2,31.354344,27.237316
3,-0.857170,134.067035
4,39.085412,-27.997393
5,31.603043,34.763726
6,43.619440,122.265737
7,41.243238,119.409787
8,-23.057488,-54.193132
9,37.906377,23.011527


In [52]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_template)

# Assign the marker layer to a variable
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))